# Belastingen uit matroos

## 1. Voorbeeld met CSV
Lees de configuratie in:

Dit leest een configuratie bestand `belasting_noos_matroos_config.yaml` in. 
```yaml
GlobalVariables:
    rootdir: "data_sets" 
    moments: [-24,0,24,48]  

    LoadsMatroos:
        website: "noos" 
        model: "observed"
        parameters: ["waterlevel"]

DataAdapter: 
    default_options:
        csv:
            sep: ","

    BelastingLocaties: 
        type: csv
        path: "belastingen_matroos.csv"
    Waterstanden: 
        type: csv
        path: "waterstanden.csv"

```

In [ ]:
from pathlib import Path

from toolbox_continu_inzicht import Config

test_data_sets_path = Path.cwd() / "data_sets"
config = Config(config_path=test_data_sets_path / "loads_noos_matroos_config.yaml")
config.lees_config()

Zet de data adapter klaar en geef deze de configuratie mee 

In [ ]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=config)

Run de module met CSV

In [ ]:
from toolbox_continu_inzicht.loads import LoadsMatroos

matroos_obs = LoadsMatroos(data_adapter=data)

In [ ]:
matroos_obs.df_in

In [ ]:
from toolbox_continu_inzicht.loads.loads_matroos.get_matroos_locations import (
    get_matroos_locations,
    get_matroos_locations_map,
)

In [ ]:
get_matroos_locations(source="observed", parameter="waterlevel")

In [ ]:
get_matroos_locations_map(source="observed", parameter="waterlevel")

In [ ]:
ds = matroos_obs.run(input="BelastingLocaties", output="Waterstanden")

In [ ]:
df = matroos_obs.df_out

In [ ]:
df

In [ ]:
df_plot = df[df["measurement_location_code"] == "Den Helder"]

In [ ]:
import numpy as np  # type: ignore

values_nan = df_plot[df_plot["value"] == -999].index
for val in values_nan:
    df_plot.loc[val, "value"] = np.nan
df_plot = df_plot.set_index("date_time")

In [ ]:
df_plot[["value"]].plot()

### Dit waren alleen metingen, herhaal nu met dataset met ook voorspellingen

Dit leest een configuratie bestand `belasting_noos_matroos_config_fcst.yaml` in. 
```yaml
GlobalVariables:
    rootdir: "data_sets" 
    moments: [-24,0,24,48] 

    LoadsMatroos:
        website: "noos"
        model: "dcsm6_kf"
        parameters: ["waterlevel"]

DataAdapter: 
    default_options:
        csv:
            sep: ","

    BelastingLocaties: 
        type: csv
        path: "belastingen_matroos.csv"
    Waterstanden: 
        type: csv
        path: "waterstanden.csv"

```

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_noos_matroos_fcst_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

matroos_fsct = LoadsMatroos(data_adapter=data)

In [ ]:
matroos_fsct.run(input="BelastingLocaties", output="Waterstanden")

In [ ]:
df_fcst = matroos_fsct.df_out
df_plot_fcst = df_fcst[
    df_fcst["measurement_location_description"] == "Hoek van Holland"
]
values_nan = df_plot_fcst[df_plot_fcst["value"] == 999].index
for val in values_nan:
    df_plot_fcst.loc[val, "value"] = np.nan
df_plot_fcst = df_plot_fcst.set_index("date_time")

In [ ]:
df_plot_fcst[["value"]].plot()

##### vergelijk de twee 

In [ ]:
ax = df_plot_fcst[["value"]].plot()
df_plot[["value"]].plot(ax=ax)
ax.legend(["dscm6", "observed"]);

#### werkt ook met vitaal.matroos.rws
werkt alleen als je in de `.env` username & password mee geeft
```yaml
vitaal_user: "..."
vitaal_password: "..."
```

en de website aanpassen

```yaml
LoadsMatroos:
        website: "vitaal" 


```

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_vitaal_matroos_fcst_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

vitaal_fsct = LoadsMatroos(data_adapter=data)

In [ ]:
vitaal_fsct.run(input="BelastingLocaties", output="Waterstanden")

In [ ]:
df_vitaal_fcst = vitaal_fsct.df_out
df_plot_vitaal = df_vitaal_fcst[
    df_vitaal_fcst["measurement_location_description"] == "Hoek van Holland"
]
values_nan = df_plot_vitaal[df_plot_vitaal["value"] == 999].index
for val in values_nan:
    df_plot_vitaal.loc[val, "value"] = np.nan
df_plot_vitaal = df_plot_vitaal.set_index("date_time")

in dit geval indentiek:

In [ ]:
ax = df_plot_fcst[["value"]].plot()
df_plot_vitaal["value"].plot(ls="--", ax=ax)

In [ ]:
from toolbox_continu_inzicht.loads import get_matroos_locations

In [ ]:
gdf = get_matroos_locations(parameter="waterlevel", source="dcsm6_kf")

In [ ]:
gdf

In [ ]:
from toolbox_continu_inzicht.loads.loads_matroos.get_matroos_locations import (
    get_matroos_sources,
)

In [ ]:
df = get_matroos_sources()

In [ ]:
df[["source_label"]]

In [ ]:
get_matroos_locations_map(source="dcsm6_kf", parameter="waterlevel")